# Descriptor In Use

- Create a n dimension Point class
- Create a Polygon class that contains a sequence point objects 


### a. N dimension point class

In [1]:
class Point:

    def __init__(self, *xi):
        self.xi = list(xi)

    def __repr__(self):
        cls_name = type(self).__name__
        coord = ', '.join(f"x{i}={x}" for i, x in enumerate(self.xi))
        return f"{cls_name}({coord})"

    def __iter__(self):
        return (x for x in self.xi)
    
    def __eq__(self, other):
        if not isinstance(other, type(self)):
            raise TypeError(f"{other} must be of type {type(self).__name__}")
        if self.dimension != other.dimension:
            raise ValueError(f"Incorrect dimension: dim(p1)={len(self.xi)} != dim(p2)={len(other.xi)}")
        return self.xi == other.xi

    def __hash__(self):
        return hash(tuple(self.xi))

    @property
    def dimension(self):
        return len(self.xi)


In [2]:
p1 = Point(0,1,2,8,4)
p1

Point(x0=0, x1=1, x2=2, x3=8, x4=4)

In [3]:
p2 = Point(1,2,8,4)
p2

Point(x0=1, x1=2, x2=8, x3=4)

In [4]:
try:
    p1 == p2
except ValueError as err:
    print(err)

Incorrect dimension: dim(p1)=5 != dim(p2)=4


### b. Polygon

In [5]:
class PointSequence:

    def __init__(self, min_length=None, max_length=None):
        self.min_length = min_length
        self.max_length = max_length
    
    def __set_name__(self, instance, name):
        self.name = name

    def __get__(self, instance, owner):
        if instance is None:
            return self
        return instance.__dict__[self.name]

    def __set__(self, instance, value):
        for element in value:
            if not isinstance(element, Point):
                raise TypeError(f"{element} must be of type Point.")
            if value.count(element) > 1:
                raise ValueError(f"{element} already in point list.")

        if self.min_length is not None and len(value) < self.min_length:
            raise ValueError(f"Below min length of {self.min_length}")
        if self.max_length is not None and len(value) > self.max_length:
            raise ValueError(f"Above max length of {self.max_length}")
        instance.__dict__[self.name] = list(value)

In [6]:
class Polygon:

    vertices = PointSequence(min_length=2, max_length=None)

    def __init__(self, *points):
        self.vertices = points
    
    def __repr__(self):
        cls_name = type(self).__name__
        return f"{cls_name}(n={len(self)}, dim={self.dimension})"

    def __len__(self):
        return len(self.vertices)

    def __contains__(self, point):
        return point in self.vertices
    
    @property
    def dimension(self):
        return self.vertices[0].dimension
    
    def append(self, point):
        if not isinstance(point, Point):
            raise TypeError("Cannot append. Must be a Point object.")
        max_length = type(self).vertices.max_length
        if len(self.vertices) == max_length:
            raise ValueError("Cannot append element, max length reached.")
        self.vertices.append(point)


In [7]:
try:
    p = Polygon(Point(1,2), Point(1,2))
except ValueError as err:
    print(err)

Point(x0=1, x1=2) already in point list.


In [8]:
try:
    p = Polygon(Point(1,2), Point(4,2), 58)
except TypeError as err:
    print(err)

58 must be of type Point


In [9]:
try:
    p = Polygon(Point(1,2), Point(4,2), Point(4,5,3))
except ValueError as err:
    print(err)

Incorrect dimension: dim(p1)=3 != dim(p2)=2


In [10]:
p =  Polygon(Point(2,2), Point(1,2))
p

Polygon(n=2, dim=2)

In [11]:
try:
    p.append(5)
except TypeError as err:
    print(err)

Cannot append. Must be a Point object.


In [12]:
p.append(Point(1,2))
p

Polygon(n=3, dim=2)

### c. Subclass Polygon

In [13]:
class Triangle(Polygon):
    vertices = PointSequence(min_length=3, max_length=3)

In [14]:
try:
    t = Triangle(Point(1,1), Point(1,2), Point(3,1), Point(4,1))
except ValueError as err:
    print(err)

Above max length of 3


In [15]:
t = Triangle(Point(1,1), Point(1,2), Point(3,1))
t

Triangle(n=3, dim=2)

In [16]:
class Rectangle(Polygon):
    vertices = PointSequence(min_length=4, max_length=4)

In [17]:
r = Rectangle(Point(1,1), Point(1,2), Point(3,1), Point(4,1))
r

Rectangle(n=4, dim=2)